In [26]:
from PyNite import FEModel3D
from PyNite.Visualization import render_model
from math import sqrt
from math import tan

-6.4053311966462765

In [49]:
def frame_displ_y(H, L, w, ang):
    """
    H, l, w, ang
    """
    
    frame_model = FEModel3D() # Creates an empty model
    H  # mm
    L  # mm
    w  # N/mm
    ang # rad

    frame_model.add_node("node1",0,0,0)
    frame_model.add_node("node2",0,H,0)
    frame_model.add_node("node3",L/2,H+(L/2)*tan(ang),0)
    frame_model.add_node("node4",L,H,0)
    frame_model.add_node("node5",L,0,0)

    m = (L/2)**2 + 500**2
    member_length = sqrt(m)

    frame_model.add_material(name='Steel', E=200e3, G=77e3, nu=0.3, rho=7.7e-6)
    frame_model.add_member('M1', 'node1', 'node2', 'Steel', 100e6, 800e6, 100e3, 2000)
    frame_model.add_member('M2', 'node2', 'node3', 'Steel', 100e6, 800e6, 100e3, 2000)
    frame_model.add_member('M3', 'node3', 'node4', 'Steel', 100e6, 800e6, 100e3, 2000)
    frame_model.add_member('M4', 'node4', 'node5', 'Steel', 100e6, 800e6, 100e3, 2000)

    frame_model.add_load_combo('L',{"D":1})
    frame_model.add_member_dist_load("M1", "Fy", -w, -w, 0, H, case="D")
    frame_model.add_member_dist_load("M4", "Fy", w, w, 0, H, case="D")
    frame_model.add_member_dist_load("M2", "Fy", -w, -w, 0, member_length, case="D")
    frame_model.add_member_dist_load("M3", "Fy", -w, -w, 0, member_length, case="D")

    frame_model.def_support('node1', True, True, True, True, True, True)
    frame_model.def_support('node5', True, True, True, True, True, True)
    #render_model(frame_model, combo_name="L", annotation_size=300)
    frame_model.analyze()
    #frame_model.Members['M1'].plot_deflection('dy', combo_name="L",n_points=1000)
    displ_1 = frame_model.Members['M1'].min_deflection('dy', combo_name="L")
    displ_2 = frame_model.Members['M2'].min_deflection('dy', combo_name="L")
    displ_3 = frame_model.Members['M3'].min_deflection('dy', combo_name="L")
    displ_4 = frame_model.Members['M4'].min_deflection('dy', combo_name="L")
    displ = min(displ_1,displ_2, displ_3, displ_4)
    return abs(displ)

In [50]:
frame_displ_y(1000, 1919, 120, 15)

0.3003017690204799

In [51]:
for L in range (1000, 1020):
    ddd = frame_displ_y(1000, L, 120, 15)
    print (ddd)

0.19790364741429373
0.1980157760472308
0.19812786019739864
0.19823989986057347
0.19835189503243678
0.1984638457085782
0.19857575188448642
0.19868761355556347
0.1987994307171124
0.19891120336434412
0.19902293149237615
0.1991346150962363
0.19924625417085579
0.19935784871107512
0.19946939871164507
0.1995809041672214
0.19969236507237126
0.1998037814215704
0.19991515320920444
0.20002648042956814


In [54]:
L = 1000
ddd = 0
while ddd <=0.8:
    ddd = frame_displ_y(1000, L, 120, 15)
    L+=1
print (L, ddd)
    

3106 0.8003411134932514
